# "Vespa submission to round 1 of TREC-COVID"
> "Reproducing the submission with Vespa python API"

- toc: false 
- badges: true
- comments: true
- categories: [COVID-19, vespa]
- image: images/coronavirus.jpg


In [1]:
# hide
%load_ext autoreload
%autoreload 2

This piece will reproduce a possible round 1 TREC-COVID submission generated with the [cord19.vespa.ai](cord19.vespa.ai) application. 

![](images/coronavirus.jpg)

## Vespa

Connect to the [CORD-19 Vespa](https://cord19.vespa.ai/) API.

In [2]:
from vespa.application import Vespa

app = Vespa(url = "https://api.cord19.vespa.ai")

Define the query model used for the submission.

In [3]:
from vespa.query import Query, OR, RankProfile

query_model = Query(
    match_phase = OR(),
    rank_profile = RankProfile(name="bm25t5")
)

## Submission

Load the topics provided by the organizers.

In [4]:
import requests
import json

topics = json.loads(requests.get("https://thigm85.github.io/data/covid19/topics-annotated.json").text)

Generate the submissions by querying the Vespa application, and organizing the results according to the TREC output format. We only return 2 hits for each request as an example. Feel free to change that to 1000 when generating your own submmission.

In [5]:
from pandas import DataFrame

submission = []
for t in topics:
    id = t['id']
    question = t['question']
    query = t['query']
    narrative = t['narrative']

    query = question + ' ' + query + ' ' + narrative 
    result = app.query(
          query=query, 
          query_model=query_model, 
          hits = 2, 
          model = {'defaultIndex': 'allt5'}, 
          summary = 'default',
          timeout = '15s',
          collapsefield = 'cord_uid',
          bolding = 'false'
      )

    i = 0
    for h in result['root']['children']:
        i+=1       
        submission.append(
            {"topicid": id,
             "Q0": "Q0",
             "docid": h["fields"].get('cord_uid'),
             "rank": i,
             "score": h['relevance'],
             "run-tag": query_model.rank_profile.name
            })

submission = DataFrame.from_records(submission)

In [6]:
submission

,topicid,Q0,docid,rank,score,run-tag
0,1,Q0,lfndq85x,1,70.993937,bm25t5
1,1,Q0,z14rf85c,2,70.613335,bm25t5
2,2,Q0,exqza1kg,1,92.731966,bm25t5
3,2,Q0,r9scxa76,2,88.443326,bm25t5
4,3,Q0,rq5nqm92,1,80.897894,bm25t5
...,...,...,...,...,...,...
65,33,Q0,79yna07e,2,96.310245,bm25t5
66,34,Q0,gd5btv69,1,81.606707,bm25t5
67,34,Q0,8p9d1c9k,2,76.193660,bm25t5
68,35,Q0,6xkm2j0f,1,109.850018,bm25t5
